## Plan
First use 1 hour resolution data, run a fourier transform and use a simple decision tree. <br>
Maybe even simple linear regression using only the n largest spikes in the fourier.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import tree
from matplotlib import pyplot as plt

C:\Users\bfesc\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.read_csv('C:\\Users\\bfesc\\Documents\\Master_thesis\\master_data\\dataport\\dataport-1hr-full.csv', 
                header = 0, index_col = 0)
data.index = data.index.map(lambda x: x[:-3])
data.index = pd.to_datetime(data.index)
data.head()

,dataid,grid
localminute,,
2017-01-01,26,1.788783
2017-01-01,9938,1.064767
2017-01-01,59,0.805333
2017-01-01,9647,1.081900
2017-01-01,93,0.916167


In [3]:
houses = pd.DataFrame(index = data[data['dataid'] == 26].index)
houses_fft = pd.DataFrame()

for i in np.unique(data['dataid']):
    if len(data[data['dataid'] == i]['grid']) == len(houses.index):
        houses[str(i)] = data[data['dataid'] == i]['grid']
        houses_fft[str(i)] = np.fft.fft(houses[str(i)])
    
houses.head()

,26,59,86,93,94,114,171,232,252,370,...,9776,9875,9932,9934,9935,9937,9938,9939,9971,9983
localminute,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00,1.788783,0.805333,0.683733,0.916167,2.054933,0.552000,0.456217,0.569400,1.218867,0.862550,...,2.136917,0.894183,6.523617,0.670233,1.666183,0.274333,1.064767,1.158533,0.348083,1.946800
2017-01-01 01:00:00,1.223650,0.802317,0.546183,0.459600,1.762017,0.394583,0.447167,0.581833,1.256733,4.089967,...,0.847483,0.737950,6.913767,0.262100,1.083350,0.245883,0.934550,1.374417,0.266783,0.444617
2017-01-01 02:00:00,0.667667,0.459883,0.425467,0.352217,1.170683,0.321000,0.292850,0.575683,1.358817,0.767333,...,0.818467,0.687533,1.616400,0.327267,0.983483,0.245683,0.946933,1.157917,0.274350,0.462567
2017-01-01 03:00:00,0.477950,0.488367,0.445467,0.321100,0.876433,0.305867,0.358683,0.557833,1.298883,0.498483,...,0.838250,0.686200,0.427600,0.670867,1.081733,0.255217,0.979700,1.008100,0.242517,0.302167
2017-01-01 04:00:00,0.414017,0.463400,0.429967,0.425067,1.131033,3.525317,0.510967,0.560133,1.354433,0.489683,...,1.319850,0.729500,0.398100,0.532500,1.008100,0.252750,0.959883,1.048167,0.251650,0.299300


In [27]:
no_components = 15
aggregate_fft = houses_fft.sum(axis=1)

f = np.argsort(abs(aggregate_fft**2))[-no_components:]
f[0] = 52 #add the week frequency, it appears further down the component list (~25)

reduced_houses = houses_fft.loc[f]
reduced_houses = reduced_houses.T #swap columns and rows so the predictors are the columns
reduced_houses.index = reduced_houses.index.astype(int)
N = len(reduced_houses.index)

reduced_houses = reduced_houses**2

reduced_houses.head()

,8026,728,8752,2,8025,729,8390,364,730,8024,8753,1,8389,365,0,52
26,(-67823.2701688-227758.18924j),(-67823.2701688+227758.18924j),(27527.4732535+578557.229935j),(27527.4732535-578557.229935j),(-14872.6645996-119036.434223j),(-14872.6645996+119036.434223j),(261577.785735-2450593.3486j),(261577.785735+2450593.3486j),(-96643.0384601+217165.145714j),(-96643.0384601-217165.145714j),(5478610.52056+5690723.71665j),(5478610.52056-5690723.71665j),(5301484.78618-6046979.11359j),(5301484.78618+6046979.11359j),(127261307.327+0j),(179769.259794+261064.327472j)
59,(35490.0125543+79546.5842125j),(35490.0125543-79546.5842125j),(5595629.32048+3839099.78504j),(5595629.32048-3839099.78504j),(243636.89365+330104.112378j),(243636.89365-330104.112378j),(-1737679.90361+85250.6133162j),(-1737679.90361-85250.6133162j),(474558.600048-416172.555477j),(474558.600048+416172.555477j),(-594514.748545-559527.50058j),(-594514.748545+559527.50058j),(-2371639.11231+7927292.70748j),(-2371639.11231-7927292.70748j),(201263743.626+0j),(-14865.3512882+15365.4500808j)
86,(-139181.649588-649087.959217j),(-139181.649588+649087.959217j),(-598850.507882-236852.768171j),(-598850.507882+236852.768171j),(-864665.708178-483552.472744j),(-864665.708178+483552.472744j),(-1351869.48253-59590.6028047j),(-1351869.48253+59590.6028047j),(-1178704.80668+2017836.50485j),(-1178704.80668-2017836.50485j),(6479258.08928+5579751.0573j),(6479258.08928-5579751.0573j),(-2952603.62045+3589449.14437j),(-2952603.62045-3589449.14437j),(175187078.431+0j),(-4274.77357891-6408.74137968j)
93,(-479905.843498-276934.349499j),(-479905.843498+276934.349499j),(1154636.14619+1239869.23369j),(1154636.14619-1239869.23369j),(-6334117.26414-1476982.70342j),(-6334117.26414+1476982.70342j),(8313494.49336+1854611.41311j),(8313494.49336-1854611.41311j),(-9499945.29455+1183279.50753j),(-9499945.29455-1183279.50753j),(2449658.84331+2690185.80586j),(2449658.84331-2690185.80586j),(34549807.0595+31039900.022j),(34549807.0595-31039900.022j),(3607969.05906+0j),(459924.91752+451479.343848j)
94,(52175.0507668+585.080582049j),(52175.0507668-585.080582049j),(2489.54734887+228363.523838j),(2489.54734887-228363.523838j),(30761.513295+98302.9930488j),(30761.513295-98302.9930488j),(127025.212501-51655.1346413j),(127025.212501+51655.1346413j),(-102577.60373-101632.676396j),(-102577.60373+101632.676396j),(1220250.85872+966013.058669j),(1220250.85872-966013.058669j),(617318.300932-80143.1851859j),(617318.300932+80143.1851859j),(41770642.1699+0j),(273621.214172+16077.4002269j)


In [5]:
response_data = pd.read_csv('C:\\Users\\bfesc\\Documents\\Master_thesis\\master_data\\dataport\\dataport-metadata.csv', 
                header = 0, index_col = 0)

response_data.head()

,active_record,building_type,program_579,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,city,...,refrigerator1,refrigerator2,security1,shed1,sprinkler1,utilityroom1,venthood1,waterheater1,waterheater2,winecooler1
dataid,,,,,,,,,,,,,,,,,,,,,
2836,NaN,Single-Family Home,NaN,NaN,yes,NaN,NaN,NaN,NaN,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2743,NaN,Single-Family Home,NaN,NaN,yes,NaN,NaN,NaN,NaN,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5323,NaN,Single-Family Home,NaN,yes,yes,NaN,NaN,NaN,NaN,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8560,NaN,Single-Family Home,NaN,yes,yes,NaN,NaN,NaN,NaN,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3313,NaN,Single-Family Home,NaN,yes,yes,NaN,NaN,NaN,NaN,Austin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
pv_houses = response_data[response_data['pv'] == 'yes'].index
ev_houses = response_data[response_data['car1'] == 'yes'].index

pv_response = np.zeros(N)
ev_response = np.zeros(N)
for i in range(N):
    if int(reduced_houses.index[i]) in pv_houses:
        pv_response[i] += 1
    if int(reduced_houses.index[i]) in ev_houses:
        ev_response[i] += 1

#reduced_houses['PV'] = pv_responsereduced_houses['EV'] = ev_response

In [28]:
model_lr = LinearRegression()
model_lr.fit(reduced_houses, pv_response)
model_lr.coef_

C:\Users\bfesc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


array([ -1.46610316e+04,   1.46610316e+04,  -1.98978675e+03,
         1.98978675e+03,  -2.48532953e+04,   2.48532953e+04,
         7.41247779e+04,  -7.41247779e+04,   2.63854456e+03,
        -2.63854456e+03,   6.13839229e+03,  -6.13839229e+03,
         7.18619043e+05,  -7.18619043e+05,  -8.29459168e-10,
         6.95408744e-08])

In [31]:
model_pv_ols = sm.OLS(pv_response,reduced_houses.astype(np.float64))
fit = model_pv_ols.fit()
fit.summary()

C:\Users\bfesc\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:703: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.661
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     44.64
Date:                Mon, 19 Feb 2018   Prob (F-statistic):           9.84e-44
Time:                        10:47:19   Log-Likelihood:                -138.74
No. Observations:                 215   AIC:                             295.5
Df Residuals:                     206   BIC:                             325.8
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
8026        2.344e-08   3.54e-08      0.663      0.508   -4.63e-08    9.32e-08
728         2.344e-08   3.54e-08      0.663      0.508   -4.63e-08    9.32e-08
8752        1.309e-08    5.4e-09      2.424      0.016    2.45e-09    2.37e-08
2           1.309e-08    5.4e-09      2.424      0.016    2.45e-09    2.37e-08
8025         1.41e-08   1.86e-08      0.756      0.450   -2.27e-08    5.09e-08
729          1.41e-08   1.86e-08      0.756      0.450   -2.27e-08    5.09e-08
8390       -1.058e-09   4.45e-09     -0.238      0.812   -9.83e-09    7.71e-09
364        -1.058e-09   4.45e-09     -0.238      0.812   -9.83e-09    7.71e-09
730        -5.342e-08   1.23e-08     -4.360      0.000   -7.76e-08   -2.93e-08
8024       -5.342e-08   1.23e-08     -4.360      0.000   -7.76e-08   -2.93e-08
8753        2.263e-09   2.27e-09      0.995      0.321   -2.22e-09    6.75e-09
1           2.263e-09   2.27e-09      0.995      0.321   -2.22e-09    6.75e-09
8389        4.236e-09   1.66e-09      2.548      0.012    9.58e-10    7.51e-09
365         4.236e-09   1.66e-09      2.548      0.012    9.58e-10    7.51e-09
0            2.38e-10   2.53e-10      0.941      0.348    -2.6e-10    7.36e-10
52          3.708e-07   1.58e-07      2.347      0.020    5.93e-08    6.82e-07
==============================================================================
Omnibus:                        2.272   Durbin-Watson:                   1.706
Prob(Omnibus):                  0.321   Jarque-Bera (JB):                2.217
Skew:                          -0.003   Prob(JB):                        0.330
Kurtosis:                       3.497   Cond. No.                     3.68e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.84e-17. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [32]:
model_ev_ols = sm.OLS(ev_response,reduced_houses.astype(np.float64))
fit = model_ev_ols.fit()
fit.summary()

C:\Users\bfesc\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:703: ComplexWarning: Casting complex values to real discards the imaginary part
  return arr.astype(dtype)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.181
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     5.053
Date:                Mon, 19 Feb 2018   Prob (F-statistic):           3.61e-06
Time:                        10:47:45   Log-Likelihood:                -110.61
No. Observations:                 215   AIC:                             239.2
Df Residuals:                     206   BIC:                             269.6
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
8026       -4.851e-09    3.1e-08     -0.156      0.876    -6.6e-08    5.63e-08
728        -4.851e-09    3.1e-08     -0.156      0.876    -6.6e-08    5.63e-08
8752        2.027e-10   4.74e-09      0.043      0.966   -9.14e-09    9.55e-09
2           2.027e-10   4.74e-09      0.043      0.966   -9.14e-09    9.55e-09
8025        1.504e-08   1.64e-08      0.919      0.359   -1.72e-08    4.73e-08
729         1.504e-08   1.64e-08      0.919      0.359   -1.72e-08    4.73e-08
8390       -2.447e-09    3.9e-09     -0.627      0.531   -1.01e-08    5.25e-09
364        -2.447e-09    3.9e-09     -0.627      0.531   -1.01e-08    5.25e-09
730         -1.89e-08   1.08e-08     -1.757      0.080   -4.01e-08     2.3e-09
8024        -1.89e-08   1.08e-08     -1.757      0.080   -4.01e-08     2.3e-09
8753       -5.991e-10      2e-09     -0.300      0.764   -4.53e-09    3.33e-09
1          -5.991e-10      2e-09     -0.300      0.764   -4.53e-09    3.33e-09
8389         1.49e-09   1.46e-09      1.021      0.308   -1.39e-09    4.37e-09
365          1.49e-09   1.46e-09      1.021      0.308   -1.39e-09    4.37e-09
0           4.421e-10   2.22e-10      1.993      0.048    4.75e-12    8.79e-10
52         -1.449e-08   1.39e-07     -0.104      0.917   -2.88e-07    2.59e-07
==============================================================================
Omnibus:                       45.204   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               68.150
Skew:                           1.364   Prob(JB):                     1.59e-15
Kurtosis:                       3.405   Cond. No.                     3.68e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.84e-17. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [37]:
n_folds = 5
fold_size = int(N/n_folds)
folds = np.random.permutation(np.arange(N)).reshape((n_folds,fold_size))
errors = np.zeros(n_folds)

for i in range(n_folds):
    test = folds[i]
    train = np.delete(folds,i,0).flatten()
    pv_tree = tree.DecisionTreeClassifier()
    fit = pv_tree.fit(X=reduced_houses.iloc[train],y=pv_response[train])
    errors[i] = np.sum((pv_tree.predict(reduced_houses.iloc[test]) - pv_response[test])**2)/fold_size # generalization error
    
print("CV generalization error", 1 - errors.sum()/n_folds)


CV generalization error 0.911627906977


C:\Users\bfesc\Anaconda3\lib\site-packages\sklearn\utils\validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


In [36]:
y_hat = np.zeros(N)

for i in range(N):
    if houses.T.iloc[i].where(houses.T.iloc[i] < 0).any():
        y_hat[i] += 1

1 - np.sum((y_hat - pv_response)**2)/N

0.96744186046511627

It appear that thresholding for signals less than 0 is the superior way of discovering photovoltaics.